In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from dataset.dataset import NoCDataset
from model.vanilla import VanillaModel

from tqdm import tqdm

In [2]:
dataset = NoCDataset()
print(f"#Samples = {len(dataset)}")
print(dataset[0])

# TODO: use advanced dgl.dataloader instead of manually dividing dataset
num_training = 0.9 * len(dataset)


#Samples = 559
(Graph(num_nodes={'packet': 10, 'router': 12},
      num_edges={('packet', 'pass', 'router'): 46, ('router', 'backpressure', 'router'): 16, ('router', 'connect', 'router'): 16, ('router', 'transfer', 'packet'): 46},
      metagraph=[('packet', 'router', 'pass'), ('router', 'router', 'backpressure'), ('router', 'router', 'connect'), ('router', 'packet', 'transfer')]), tensor([1057.8928,  887.8571]))


In [3]:
device = "cpu"
# device = "cuda:0"

model = VanillaModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
losses = []

epoches = 100
for e in tqdm(range(epoches)):
    for i, data in enumerate(dataset):
        if i > num_training:
            break

        g, congestion = data
        g = g.to(device)

        pred = model(g)
        loss = F.mse_loss(pred, torch.log(congestion+1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f"iteration: {i}; loss = {loss}")
            losses.append(loss.item())
            # print(f"pred = {pred}")
            # print(f"congestion = {congestion}")
            
plt.ylim(0, 20)
plt.plot(losses)


  0%|          | 0/100 [00:00<?, ?it/s]

iteration: 0; loss = 74.66802978515625
iteration: 100; loss = 32.879417419433594
iteration: 200; loss = 14.707178115844727
iteration: 300; loss = 6.074963569641113
iteration: 400; loss = 6.498138427734375


  1%|          | 1/100 [00:02<03:33,  2.16s/it]

iteration: 500; loss = 7.965472221374512
iteration: 0; loss = 8.72354793548584
iteration: 100; loss = 13.729049682617188
iteration: 200; loss = 7.677690505981445
iteration: 300; loss = 5.306740760803223
iteration: 400; loss = 5.947522163391113


  2%|▏         | 2/100 [00:04<03:32,  2.16s/it]

...

 31%|███       | 31/100 [01:07<02:29,  2.17s/it]

iteration: 500; loss = 1.4769532680511475
iteration: 0; loss = 1.3258411884307861
iteration: 100; loss = 1.2902731895446777


 31%|███       | 31/100 [01:08<02:32,  2.21s/it]

iteration: 200; loss = 9.56175422668457


KeyboardInterrupt: 

In [4]:
# test accuracy
# we use relative error to measure

mae_losses = []

for i, data in enumerate(dataset):
    if i <= num_training:
        continue

    g, congestion = data
    g = g.to(device)

    pred = model(g)
    loss = F.mse_loss(pred, torch.log(congestion+1))
    pred_congestion = torch.exp(pred) - 1

    print(f"iteration: {i}; loss = {loss.item()}")
    print(f"pred = {pred_congestion.detach().numpy()}")
    print(f"congestion = {congestion.numpy()}")

    mae_losses.append(F.l1_loss(pred_congestion, congestion))

print("*" * 30)
mae_losses = torch.stack(mae_losses).mean()
print(f"average mae loss: {mae_losses}")



iteration: 504; loss = 2.2492332458496094
pred = [224.05904 233.71704]
congestion = [1118.0714  939.2857]
iteration: 505; loss = 4.427812099456787
pred = [10.185406   4.6935515]
congestion = [0. 0.]
iteration: 506; loss = 3.8778021335601807
pred = [112.22044 120.26068]
congestion = [838.12085 838.4623 ]
iteration: 507; loss = 10.677654266357422
pred = [ 0.35073853 -0.4048471 ]
congestion = [132.3   0. ]
iteration: 508; loss = 24.919662475585938
pred = [149.622   164.65103]
congestion = [0.15441176 0.        ]
iteration: 509; loss = 16.004436492919922
pred = [222.99495 123.44561]
congestion = [4305.2856    0.    ]
iteration: 510; loss = 11.50244426727295
pred = [40.69868  19.383211]
congestion = [0. 0.]
iteration: 511; loss = 6.040027618408203
pred = [9.872671 4.655108]
congestion = [88.588036 88.588036]
iteration: 512; loss = 2.320106029510498
pred = [51.274147 53.530777]
congestion = [247.46443 240.16798]
iteration: 513; loss = 4.5643157958984375
pred = [8.413722 6.577396]
congestion 